# Treinamento de Modelo de Machine Learning usando AWS SageMaker

### Membros
* 2315530 - Clairton Carneiro Luz

## 1. Introdução

Este documento apresenta um guia para o treinamento de um modelo de Machine Learning utilizando o AWS SageMaker. O objetivo desta atividade é permitir a prática na implementação de um pipeline de aprendizado de máquina, desde a preparação do ambiente até a implantação do modelo treinado.


## 2. Objetivo

O objetivo desta atividade é desenvolver habilidades na utilização do AWS SageMaker para treinar e implantar modelos de Machine Learning. O processo envolve a criação de um ambiente adequado, manipulação de dados, treinamento do modelo, avaliação de desempenho e implantação para uso em produção.

## 3. Preparação do Ambiente:

   ### 3.1 Criar um notebook instance no SageMaker.

* Acesse o console AWS e vá para o serviço `Amazon SageMaker AI`.

* No painel esquerdo, clique em `Notebooks`.
* Clique em `Create notebook instance`.
* Defina um nome para a instância e escolha um tipo adequado, como ml.t3.medium.
* Em IAM role, selecione Create a new role, permitindo acesso ao Amazon S3.
* Clique em Create notebook instance e aguarde até que o status mude para InService.

* Agora faça o upload desse notebook para que possa executa-lo no ambiente em nuvem.

### 3.2 Configurar as permissões IAM para permitir o acesso ao S3 e SageMaker.

* No console AWS, vá para o serviço IAM.

* Clique em Roles e selecione a função atribuída ao notebook SageMaker.
* Em Permissões, adicione as políticas AmazonS3FullAccess e AmazonSageMakerFullAccess.
* Salve as alterações para garantir que o SageMaker possa acessar os recursos necessários.

### 3.3 Instalar as bibliotecas necessárias.

* Após iniciar o notebook, execute o seguinte comando para instalar as bibliotecas necessárias:

In [1]:
%pip install -q -U sagemaker pandas numpy scikit-learn kagglehub matplotlib seaborn
%pip install -q boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 4. Download e Preparação dos Dados:

### 4.1 Download dataset

In [2]:

import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("eslamelsolya/laptop-price-prediction")

print("Path to dataset files:", path)
df_original = pd.read_csv(path + "/laptop_data.csv", index_col=0)

df = df_original.copy()
df_original.head()

100%|██████████| 398k/398k [00:00<00:00, 22.9MB/s]

Extracting files...
Path to dataset files: /home/ec2-user/.cache/kagglehub/datasets/eslamelsolya/laptop-price-prediction/versions/1


,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


### 4.2 Realizar a limpeza e pré-processamento dos dados (tratamento de valores faltantes, normalização, etc.).

In [3]:
df.shape

(1303, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           1303 non-null   object 
 1   TypeName          1303 non-null   object 
 2   Inches            1303 non-null   float64
 3   ScreenResolution  1303 non-null   object 
 4   Cpu               1303 non-null   object 
 5   Ram               1303 non-null   object 
 6   Memory            1303 non-null   object 
 7   Gpu               1303 non-null   object 
 8   OpSys             1303 non-null   object 
 9   Weight            1303 non-null   object 
 10  Price             1303 non-null   float64
dtypes: float64(2), object(9)
memory usage: 122.2+ KB


In [5]:
print(df.duplicated().sum())
df.drop_duplicates(inplace = True)
df.duplicated().sum()

29


0

In [6]:
df.groupby("Company")["TypeName"].value_counts()

Company    TypeName          
Acer       Notebook               76
           2 in 1 Convertible      8
           Gaming                  8
           Netbook                 5
           Ultrabook               4
Apple      Ultrabook              21
Asus       Notebook               62
           Gaming                 54
           Ultrabook              18
           2 in 1 Convertible     13
           Netbook                 4
Chuwi      Notebook                3
Dell       Notebook              159
           Ultrabook              49
           Gaming                 40
           2 in 1 Convertible     30
           Workstation            11
           Netbook                 2
Fujitsu    Notebook                3
Google     Ultrabook               3
HP         Notebook              180
           Ultrabook              36
           2 in 1 Convertible     19
           Workstation            14
           Gaming                 12
           Netbook                 7
Huawei  

In [7]:
df.isna().sum()

Company             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price               0
dtype: int64

In [8]:
# Removendo unidades de medida e convertendo para float
print(df["Weight"].values)
df["Weight"] = df["Weight"].str.replace("kg", "")
df["Weight"] = df["Weight"].astype(float)

['1.37kg' '1.34kg' '1.86kg' ... '1.3kg' '1.5kg' '2.19kg']


In [9]:
# removendo unidades de medida e convertendo para float
print(df["Ram"].values)
df["Ram"] = df["Ram"].str.replace("GB", "")
df["Ram"] = df["Ram"].astype(float)

['8GB' '8GB' '8GB' ... '16GB' '2GB' '6GB']


In [10]:
from sklearn.preprocessing import LabelEncoder
# Convertendo colunas categoricas para numericas
label_encoder = LabelEncoder()
for col in df.select_dtypes(include = ["object"]).columns:
  df[col] = label_encoder.fit_transform(df[col])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10, 10))
sns.heatmap(df.corr(), annot = True, fmt = ".2g", cmap = "viridis")
plt.show()

## 5 Dividir o dataset em conjuntos de treino e teste.

### 5.1 Separando features e target

In [ ]:
import numpy as np

X = df.drop("Price", axis = 1)
y = np.log(df["Price"] + 1e-8)

### 5.2 Separando treino, teste e validação

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### 5.3 Transformando valores para ficar no mesmo padrão de escala.

In [ ]:
from sklearn.preprocessing import StandardScaler

Scaling_Features = StandardScaler()
X_train = Scaling_Features.fit_transform(X_train)
X_test = Scaling_Features.transform(X_test)

## 6. Armazenamento dos Dados no S3:

### 6.1 Criar um bucket S3.

* No console AWS, vá para o serviço S3.

* Clique em Create bucket e defina um nome único.

* Configure permissões adequadas e finalize a criação.


#### Outra opção é utilizar o bucket default que o sagemaker já configura pra você com o comando `sagemaker_session.default_bucket()`

### 6.2 Upload dos dados de treino, teste e validação para o bucket S3.

In [ ]:
# Salvando os dados de treino, teste e validação no s3
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
# caso queira utilizar um bucket especifico usa a linha abaixo para definir o bucket no lugar da linha acima.
# bucket = "unifor-aws-sagemake"
prefix = "laptop-price-prediction"

train_file = "train.csv"
test_file = "test.csv"

pd.concat([pd.DataFrame(y_train), pd.DataFrame(X_train)],
          axis=1).to_csv(train_file, header=False, index=False)
pd.concat([pd.DataFrame(y_test), pd.DataFrame(X_test)],
          axis=1).to_csv(test_file, header=False, index=False)

sagemaker_session.upload_data(train_file, bucket, prefix + "/train")
sagemaker_session.upload_data(test_file, bucket, prefix + "/test")

## 7. Treinamento do Modelo:

### 7.1 Escolher um algoritmo de machine learning disponível no SageMaker (por exemplo, XGBoost).

In [ ]:
import sagemaker
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator

# Defina o papel de execução do SageMaker
role = get_execution_role()

### 7.2 Configurar o job de treinamento no SageMaker.

In [ ]:
# Criar e treinar modelos
models = {
    "xgboost": Estimator(
        image_uri=sagemaker.image_uris.retrieve(
            "xgboost", sagemaker_session.boto_region_name, version="1.5-1"),
        role=role,
        instance_count=1,
        instance_type="ml.m5.large",
        output_path=f"s3://{bucket}/{prefix}/models/xgboost",
        sagemaker_session=sagemaker_session,
        hyperparameters={
            "objective": "reg:squarederror",
            "num_round": "100"
        }
    )
    # "linear": Estimator(
    #     image_uri=sagemaker.image_uris.retrieve(
    #         "linear-learner", sagemaker_session.boto_region_name),
    #     role=role,
    #     instance_count=1,
    #     instance_type="ml.m5.large",
    #     output_path=f"s3://{bucket}/{prefix}/models/linear",
    #     sagemaker_session=sagemaker_session,
    #     hyperparameters={
    #         "predictor_type": "regressor"
    #     }
    # ),
    # "random_forest": Estimator(
    #     image_uri=sagemaker.image_uris.retrieve(
    #         "randomcutforest", sagemaker_session.boto_region_name),
    #     role=role,
    #     instance_count=1,
    #     instance_type="ml.m5.large",
    #     output_path=f"s3://{bucket}/{prefix}/models/random_forest",
    #     sagemaker_session=sagemaker_session,
    #     hyperparameters={
    #         "num_samples_per_tree": "512",
    #         "num_trees": "50",
    #         "feature_dim": str(X_train.shape[1])
    #     }
    # )
}

### 7.3 Iniciar o treinamento do modelo utilizando os dados armazenados no S3.

In [ ]:
# Treinando o modelo

train_key = f"s3://{bucket}/{prefix}/train/{train_file}"
train_input = TrainingInput(
    train_key,
    content_type="text/csv"
)

# for model_name, estimator in models.items():
#     estimator.fit({"train": train_input})


In [ ]:
models["xgboost"].fit({"train": train_input})

In [ ]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = models["xgboost"].deploy(
    initial_instance_count=1, 
    serializer=CSVSerializer(),
    instance_type="ml.m5.large")

In [ ]:
models["linear"].fit({"train": train_input})

In [ ]:
models["random_forest"].fit({"train": TrainingInput(
    train_key,
    content_type="text/csv",
    distribution="ShardedByS3Key"
)})

## 8. Avaliação do Modelo:

### 8.1 Realizar previsões utilizando o conjunto de teste.

In [ ]:
# Avaliação dos modelos
from sagemaker.serializers import CSVSerializer
from sagemaker.predictor import Predictor
from sklearn.metrics import root_mean_squared_error

best_model = "xboost"
best_rmse = float("inf")
# rmse_scores = {}

# models2 = {
#     "xgboost": models["xgboost"]
# }

# for model_name, estimator in models.items():
#     print(estimator.latest_training_job.name)
#     predictor = Predictor(
#         endpoint_name=estimator.latest_training_job.name,
#         sagemaker_session=sagemaker_session,
#         serializer=CSVSerializer()
#     )


y_pred = np.array([float(xgb_predictor.predict(x)) for x in X_test])
rmse = root_mean_squared_error(y_test, y_pred)
print(f"{model_name}: RMSE = {rmse}")


In [ ]:
# imprima um grafico que mostre a curva de aprendizado do modelo mostrando a linha o rmse do treino e do teste
# conforme o numero de amostras aumenta

import matplotlib.pyplot as plt
import numpy as np

def plot_learning_curve(model, X_train, y_train, X_test, y_test):
    train_rmse = []
    test_rmse = []
    for i in range(1, len(X_train) + 1):
        model.fit(X_train[:i], y_train[:i])
        y_train_pred = model.predict(X_train[:i])
        y_test_pred = model.predict(X_test)
        train_rmse.append(np.sqrt(mean_squared_error(y_train[:i], y_train_pred)))
        test_rmse.append(np.sqrt(mean_squared_error(y_test, y_test_pred))
    plt.plot(train_rmse, label="train")
    plt.plot(test_rmse, label="test")
    plt.xlabel("Number of samples")
    plt.ylabel("RMSE")
    plt.legend()
    plt.show()
    
plot_learning_curve(models["xgboost"], X_train, y_train, X_test, y_test)



In [ ]:

import numpy as np

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

y_pred = predict(X_test)
y_pred = np.array(y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot(range(0, 20), range(0, 20), color="red")
plt.xlabel("True Price")
plt.ylabel("Predicted Price")

plt.show()

In [ ]:

s3_client = sagemaker_session.boto_session.client("s3")
response = s3_client.get_object(
    Bucket=bucket, Key=f"{prefix}/output/test.csv.out")
response_body = response["Body"].read()

predictions = pd.read_csv(io.BytesIO(response_body), header=None)
predictions = predictions.squeeze()


mean_squared_error(y_test, predictions)

### 8.2 Avaliar o desempenho do modelo utilizando métricas apropriadas (acurácia, precisão, recall, F1-score).

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2:", r2)


### 8.3 Gerar um relatório de avaliação.

In [ ]:
import boto3

# Testando o modelo
y_pred = predictor.predict(X_test)

from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)



# Salvando o modelo
model_path = sklearn.model_data
model_path

boto3.Session().resource("s3").Bucket(bucket).download_file(model_path, "model.tar.gz")

## 9. Implantação do Modelo:

### 9.1 Implantar o modelo treinado como um endpoint no SageMaker.

In [ ]:
# Deploy do modelo
predictor = sklearn.deploy(
    initial_instance_count = 1,
    instance_type = "ml.t2.medium"
)

### 9.2 Testar o endpoint realizando previsões em tempo real.

In [ ]:
# validando o modelo
y_pred = predictor.predict(X_val)

mean_squared_error(y_val, y_pred)

### 9.3 Removendo endpoint

In [ ]:
# Deletando o endpoint
predictor.delete_endpoint()